In [ ]:
'''
Bùi Thị Thanh Xuân
19110522
'''

**1. Giải thích vì sao khi dùng Naive-Learning thì xe taxi chỉ đứng yên một chỗ?**

**Trả lời:** Naive-Learning là một quá trình trong đó agent có một mục tiêu mà không cần biết cách đạt được mục tiêu nhưng lại nhận được phần thưởng từ môi trường. Cho nên khi Intialize có thể khiến taxi rơi vào vị trí không di chuyển được, nhưng reward vẫn được nhận cho action này.

**2. Giải thích vì sao khi dùng Q-Leaning thì xe taxi có thể đón và trả khách được?**

**Trả lời**:  Q-Learning tìm ra các action tối ưu ở mỗi state. Chọn action có giá trị cao cho nó khi mà nó nhìn vô cái Q-table, nếu value là 0 cho tất cả các option, chọn 1 random action, Bắt đầu với 100% là exploration và giảm dần đến 0% nó chuyển sang mode exploitation.

**3. Tìm hiểu một game khác trên OpenAI và thiết lập cho agent chơi được**


**Solving the CartPole balancing environment**

- Ý tưởng của CartPole là có một cái cột đứng trên đầu một chiếc xe đẩy. Mục tiêu là cân bằng cột này bằng cách ngọ nguậy/di chuyển xe đẩy từ bên này sang bên kia để giữ cho cột thăng bằng thẳng đứng.

- Môi trường được coi là thành công nếu chúng ta có thể cân bằng trong 200 khung hình và thất bại được coi là khi cột nghiêng hơn 15 độ so với phương thẳng đứng hoàn toàn.

- Mỗi khung hình mà chúng tôi thực hiện với cột "balanced" (dưới 15 độ so với phương thẳng đứng), "điểm số" của chúng tôi được +1 và mục tiêu của chúng tôi là số điểm 200.

- Để minh họa điều này, chúng ta sẽ bắt đầu bằng cách tạo một agent, khi ở trong môi trường cartpole, nó chỉ chọn ngẫu nhiên các actions (trái và phải). Hãy nhớ lại rằng mục tiêu của chúng tôi là đạt được số điểm 200, nhưng chúng tôi sẽ tiếp tục và sử dụng bất kỳ tình huống nào mà chúng tôi đã ghi được trên 50 điểm để học hỏi.



In [ ]:
pip install cython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%pip install scipy h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%pip install pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install -U --no-deps tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 14.7 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=9571d7d6e82b1e69bb4ac445c0cee0aa4b3d7f8e1c7459069ff2b7a348072683
  Stored in directory: /root/.cache/pip/wheels/65/9b/15/cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn


In [ ]:
pip install gym[classic_control]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.3 MB 15.3 MB/s 


In [ ]:
!pip install pygame 

import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<Surface(640x480x32 SW)>

In [ ]:
import gym
import random
import numpy as np
import tflearn

from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import median, mean
from collections import Counter

LR = 1e-3
env = gym.make("CartPole-v0")
env.reset()
goal_steps = 500
score_requirement = 50
initial_games = 10000

Instructions for updating:
non-resource variables are not supported in the long term
/usr/local/lib/python3.8/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.8/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
def some_random_games_first():
    # Each of these is its own game.
    for episode in range(5):
        env.reset()
        # this is each frame, up to 200...but we wont make it that far.
        for t in range(200):
            # This will display the environment
            # Only display if you really want to see it.
            # Takes much longer to display it.
            env.render()
            
            # This will just create a sample action in any environment.
            # In this environment, the action can be 0 or 1, which is left or right
            action = env.action_space.sample()
            
            # this executes the environment with an action, 
            # and returns the observation of the environment, 
            # the reward, if the env is over, and other info.
            observation, reward, done, info = env.step(action)
            if done:
                break

In [ ]:
def initial_population():
    # [OBS, MOVES]
    training_data = []
    # all scores:
    scores = []
    # just the scores that met our threshold:
    accepted_scores = []
    # iterate through however many games we want:
    for _ in range(initial_games):
        score = 0
        # moves specifically from this environment:
        game_memory = []
        # previous observation that we saw
        prev_observation = []
        # for each frame in 200
        for _ in range(goal_steps):
            # choose random action (0 or 1)
            action = random.randrange(0,2)
            # do it!
            observation, reward, done, info = env.step(action)
            
            # notice that the observation is returned FROM the action
            # so we'll store the previous observation here, pairing
            # the prev observation to the action we'll take.
            if len(prev_observation) > 0 :
                game_memory.append([prev_observation, action])
            prev_observation = observation
            score+=reward
            if done: break

        # IF our score is higher than our threshold, we'd like to save
        # every move we made
        # NOTE the reinforcement methodology here. 
        # all we're doing is reinforcing the score, we're not trying 
        # to influence the machine in any way as to HOW that score is 
        # reached.
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                # convert to one-hot (this is the output layer for our neural network)
                if data[1] == 1:
                    output = [0,1]
                elif data[1] == 0:
                    output = [1,0]
                    
                # saving our training data
                training_data.append([data[0], output])

        # reset env to play again
        env.reset()
        # save overall scores
        scores.append(score)
    
    # just in case you wanted to reference later
    training_data_save = np.array(training_data)
    np.save('saved.npy',training_data_save)
    
    # some stats here, to further illustrate the neural network magic!
    print('Average accepted score:',mean(accepted_scores))
    print('Median score for accepted scores:',median(accepted_scores))
    print(Counter(accepted_scores))
    
    return training_data

In [ ]:
def neural_network_model(input_size):

    network = input_data(shape=[None, input_size, 1], name='input')

    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(network, tensorboard_dir='log')

    return model


def train_model(training_data, model=False):

    X = np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]),1)
    y = [i[1] for i in training_data]

    if not model:
        model = neural_network_model(input_size = len(X[0]))
    
    model.fit({'input': X}, {'targets': y}, n_epoch=5, snapshot_step=500, show_metric=True, run_id='openai_learning')
    return model

In [ ]:
training_data = initial_population()

Average accepted score: 61.05555555555556
Median score for accepted scores: 58.0
Counter({50.0: 38, 51.0: 27, 54.0: 25, 55.0: 22, 52.0: 20, 53.0: 20, 56.0: 20, 60.0: 17, 58.0: 15, 61.0: 13, 63.0: 13, 64.0: 12, 57.0: 11, 67.0: 10, 62.0: 9, 65.0: 9, 72.0: 9, 59.0: 8, 70.0: 8, 69.0: 6, 73.0: 6, 71.0: 6, 76.0: 5, 68.0: 5, 75.0: 5, 83.0: 4, 66.0: 4, 79.0: 3, 85.0: 3, 74.0: 3, 77.0: 3, 84.0: 2, 80.0: 2, 78.0: 2, 81.0: 1, 99.0: 1, 87.0: 1, 82.0: 1, 102.0: 1, 86.0: 1, 97.0: 1, 101.0: 1, 136.0: 1, 88.0: 1, 95.0: 1, 89.0: 1, 91.0: 1})


<ipython-input-12-d18fe9924588>:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_data_save = np.array(training_data)


In [ ]:
model = train_model(training_data)

Training Step: 1774  | total loss: 0.67580 | time: 2.735s
| Adam | epoch: 005 | loss: 0.67580 - acc: 0.5855 -- iter: 22656/22701
Training Step: 1775  | total loss: 0.67540 | time: 2.741s
| Adam | epoch: 005 | loss: 0.67540 - acc: 0.5848 -- iter: 22701/22701
--


In [ ]:
scores = []
choices = []
for each_game in range(10):
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    for _ in range(goal_steps):
        env.render()

        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(model.predict(prev_obs.reshape(-1,len(prev_obs),1))[0])

        choices.append(action)
                
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        game_memory.append([new_observation, action])
        score+=reward
        if done: break

    scores.append(score)

print('Average Score:',sum(scores)/len(scores))
print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))
print(score_requirement)

Average Score: 200.0
choice 1:0.4965  choice 0:0.5035
50
